# SQLite Lab

In this lab, you'll add database persistence to your API. Instead of losing data when the server restarts, tasks will be stored in a SQLite database.

**What you'll learn:**
- Why databases matter for APIs
- SQLite basics
- Connecting FastAPI to a database
- CRUD operations with SQL

**Prerequisites:**
- Completed the FastAPI CRUD lab
- Basic understanding of SQL (helpful but not required)

**Time:** ~40 minutes

---

## Part 1: Why a Database?

In our previous lab, we stored tasks in a Python dictionary:

```python
tasks_db: dict[int, dict] = {}
```

**Problems with in-memory storage:**
- Data is lost when the server restarts
- Can't share data between multiple server instances
- Limited by available RAM

**Benefits of a database:**
- Data persists across restarts
- Can handle large amounts of data
- Supports complex queries
- Multiple servers can share the same database

### Why SQLite?

SQLite is perfect for learning and small projects:
- **No setup required** — It's just a file
- **Built into Python** — No installation needed
- **SQL compatible** — Skills transfer to other databases
- **Lightweight** — Great for development and small apps

---

## Part 2: SQLite Basics

Let's understand SQLite directly before integrating with FastAPI.

In [ ]:
import sqlite3

# Connect to database (creates file if it doesn't exist)
conn = sqlite3.connect("tasks.db")
cursor = conn.cursor()

print("Connected to SQLite database!")

In [ ]:
# Create the tasks table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS tasks (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        description TEXT,
        completed BOOLEAN DEFAULT FALSE
    )
""")
conn.commit()
print("Table created!")

In [ ]:
# CREATE - Insert a task
cursor.execute(
    "INSERT INTO tasks (title, description) VALUES (?, ?)",
    ("Learn SQL", "Understand basic database operations")
)
conn.commit()
print(f"Inserted task with ID: {cursor.lastrowid}")

In [ ]:
# READ - Get all tasks
cursor.execute("SELECT * FROM tasks")
tasks = cursor.fetchall()
print("All tasks:", tasks)

In [ ]:
# UPDATE - Mark task as completed
cursor.execute("UPDATE tasks SET completed = TRUE WHERE id = ?", (1,))
conn.commit()
print(f"Updated {cursor.rowcount} row(s)")

In [ ]:
# DELETE - Remove a task
cursor.execute("DELETE FROM tasks WHERE id = ?", (1,))
conn.commit()
print(f"Deleted {cursor.rowcount} row(s)")

In [ ]:
conn.close()
print("Connection closed")

---

## Part 3: FastAPI with SQLite

Create `main.py` with database integration:

```python
import sqlite3
from contextlib import contextmanager
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional

DATABASE_URL = "tasks.db"

@contextmanager
def get_db():
    conn = sqlite3.connect(DATABASE_URL)
    conn.row_factory = sqlite3.Row
    try:
        yield conn
    finally:
        conn.close()

def init_db():
    with get_db() as conn:
        conn.execute("""
            CREATE TABLE IF NOT EXISTS tasks (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT NOT NULL,
                description TEXT,
                completed BOOLEAN DEFAULT 0
            )
        """)
        conn.commit()

class TaskCreate(BaseModel):
    title: str
    description: Optional[str] = None

class Task(BaseModel):
    id: int
    title: str
    description: Optional[str]
    completed: bool

app = FastAPI(title="Task API with SQLite")

@app.on_event("startup")
def startup():
    init_db()

@app.post("/tasks", response_model=Task, status_code=201)
def create_task(task: TaskCreate):
    with get_db() as conn:
        cursor = conn.execute(
            "INSERT INTO tasks (title, description) VALUES (?, ?)",
            (task.title, task.description)
        )
        conn.commit()
        new_task = conn.execute(
            "SELECT * FROM tasks WHERE id = ?", 
            (cursor.lastrowid,)
        ).fetchone()
    return dict(new_task)

@app.get("/tasks", response_model=list[Task])
def list_tasks():
    with get_db() as conn:
        tasks = conn.execute("SELECT * FROM tasks").fetchall()
    return [dict(task) for task in tasks]

@app.get("/tasks/{task_id}", response_model=Task)
def get_task(task_id: int):
    with get_db() as conn:
        task = conn.execute(
            "SELECT * FROM tasks WHERE id = ?", 
            (task_id,)
        ).fetchone()
    if task is None:
        raise HTTPException(status_code=404, detail="Task not found")
    return dict(task)

@app.delete("/tasks/{task_id}", status_code=204)
def delete_task(task_id: int):
    with get_db() as conn:
        existing = conn.execute(
            "SELECT * FROM tasks WHERE id = ?", 
            (task_id,)
        ).fetchone()
        if existing is None:
            raise HTTPException(status_code=404, detail="Task not found")
        conn.execute("DELETE FROM tasks WHERE id = ?", (task_id,))
        conn.commit()
    return None
```

---

## Part 4: Testing Persistence

1. Start the server: `uvicorn main:app --reload`
2. Create tasks using `/docs`
3. Stop the server (Ctrl+C)
4. Start again
5. Tasks are still there!

### SQL Quick Reference

```sql
-- Create
INSERT INTO tasks (title) VALUES ('My Task');

-- Read
SELECT * FROM tasks;
SELECT * FROM tasks WHERE id = 1;

-- Update
UPDATE tasks SET completed = 1 WHERE id = 1;

-- Delete
DELETE FROM tasks WHERE id = 1;
```

### Prevent SQL Injection

```python
# BAD - vulnerable
cursor.execute(f"SELECT * FROM tasks WHERE title = '{user_input}'")

# GOOD - parameterized
cursor.execute("SELECT * FROM tasks WHERE title = ?", (user_input,))
```

---

## Summary

| Concept | What it does |
|---------|-------------|
| **SQLite** | Lightweight file-based database |
| **Connection** | Link between Python and database |
| **Cursor** | Executes SQL commands |
| **Parameterized queries** | Prevent SQL injection |

### Key SQL Commands

| Operation | SQL |
|-----------|-----|
| Create | `INSERT INTO table (cols) VALUES (vals)` |
| Read | `SELECT * FROM table WHERE condition` |
| Update | `UPDATE table SET col = val WHERE condition` |
| Delete | `DELETE FROM table WHERE condition` |

---

## Exercises

1. **Add search** — Find tasks by title using SQL `LIKE`
2. **Add pagination** — Use `LIMIT` and `OFFSET`
3. **Add timestamps** — Add `created_at` column
4. **Try an ORM** — Explore SQLModel or SQLAlchemy

---

## Next Steps

Check out the **[Starter Project](./starter-project/)** for a clean template for your own projects.